In [1]:
import numpy as np
import open3d as o3d
import trimesh
import pyrender

import json

from auxiliary import values as v

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
group = 'Gr4'
specimens = v.specimens[group]

Without Landmarks

In [24]:
source_path = v.data_path + f'{group}/3DShape/Tissue/myocardium/2019{specimens[0]}Shape.ply'
target_path = v.data_path + f'{group}/3DShape/Tissue/myocardium/2019{specimens[1]}Shape.ply'

# source_path = v.data_path + f'Gr1/3DShape/Tissue/myocardium/20190208_E2Shape.ply'
# target_path = v.data_path + f'Gr1/3DShape/Membrane/myocardium/20190208_E2_myocardium.ply'

source_mesh = o3d.io.read_triangle_mesh(source_path)
target_mesh = o3d.io.read_triangle_mesh(target_path)

source_pcd = source_mesh.sample_points_uniformly(number_of_points=10000)
target_pcd = target_mesh.sample_points_uniformly(number_of_points=10000)

source_pcd.scale(1 / np.max(source_pcd.get_max_bound() - source_pcd.get_min_bound()), center=source_pcd.get_center())
target_pcd.scale(1 / np.max(target_pcd.get_max_bound() - target_pcd.get_min_bound()), center=target_pcd.get_center())

# Normals estimation
radius_normal = 0.2
source_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
target_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

# FPFH features computation
radius_feature = 0.5
fpfh_source = o3d.pipelines.registration.compute_fpfh_feature(source_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
fpfh_target = o3d.pipelines.registration.compute_fpfh_feature(target_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

# RANSAC registration
best_fitness = -np.inf
best_transformation = np.eye(4)
num_trials = 20
ransac_n = 4
threshold = 0.1

for i in range(num_trials):
    ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        target_pcd, source_pcd, fpfh_target, fpfh_source, mutual_filter=False, max_correspondence_distance=threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=ransac_n,
        checkers=[o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                  o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(threshold)],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
    )

    if ransac_result.fitness > best_fitness:
        best_fitness = ransac_result.fitness
        best_transformation = ransac_result.transformation
        
print(best_transformation)
print(best_fitness)

target_pcd.transform(best_transformation)
target_mesh.transform(best_transformation)

o3d.visualization.draw_geometries([source_pcd, target_pcd])
o3d.visualization.draw_geometries([source_mesh, target_mesh])

[[ 9.90157908e-01 -1.35502221e-01 -3.50209206e-02 -1.29102790e+02]
 [ 1.27461516e-01  9.76425398e-01 -1.74203916e-01 -7.74526874e+01]
 [ 5.78003339e-02  1.68025566e-01  9.84086648e-01 -1.15501098e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9274


In [15]:
o3d.visualization.draw_geometries([source_mesh, target_mesh])

With Landmarks

Center origin

In [3]:
group = 'Gr4'
specimens = v.specimens[group]

meshes = []
for specimen in specimens:
    path = v.data_path + f'{group}/3DShape/Tissue/myocardium/2019{specimen}Shape.ply'
    mesh = o3d.io.read_triangle_mesh(path)
    meshes.append(mesh)

In [4]:
o3d.visualization.draw_geometries(meshes)

In [5]:
centered_meshes = []

for mesh in meshes:
    mesh_center = mesh.get_center()
    
    # Move mesh to origin
    trans2origin = np.eye(4)
    trans2origin[:3, 3] = -mesh_center
    
    mesh.transform(trans2origin)
    centered_meshes.append(mesh)

In [6]:
o3d.visualization.draw_geometries(centered_meshes)

Pipeline

In [2]:
group = 'Gr4'
tissue = 'myocardium'

In [3]:
import json


class MeshRegistration:
    def __init__(self, group='Gr4'):
        self.group = group
        self.specimens = v.specimens[group]
        
        self.tissue_meshes_myo = []
        self.cells_meshes_myo = []
        self.nuclei_meshes_myo = []
        
        self.tissue_meshes_spl = []
        self.cells_meshes_spl = []
        self.nuclei_meshes_spl = []
        
        self.landmarks = []
        
        self.load_meshes()
        
    def load_meshes(self):
        for specimen in self.specimens:
            tissue_path_myo = v.data_path + f'{self.group}/3DShape/Tissue/myocardium/2019{specimen}Shape.ply'
            cells_path_myo = v.data_path + f'{self.group}/3DShape/Membrane/myocardium/2019{specimen}_myocardium.ply'
            nuclei_path_myo = v.data_path + f'{self.group}/3DShape/Nuclei/myocardium/2019{specimen}_myocardium.ply'
            
            tissue_path_spl = v.data_path + f'{self.group}/3DShape/Tissue/splanchnic/2019{specimen}Shape.ply'
            cells_path_spl = v.data_path + f'{self.group}/3DShape/Membrane/splanchnic/2019{specimen}_splanchnic.ply'
            nuclei_path_spl = v.data_path + f'{self.group}/3DShape/Nuclei/splanchnic/2019{specimen}_splanchnic.ply'

            tissue_mesh_myo = o3d.io.read_triangle_mesh(tissue_path_myo)
            cells_mesh_myo = o3d.io.read_triangle_mesh(cells_path_myo)
            nuclei_mesh_myo = o3d.io.read_triangle_mesh(nuclei_path_myo)
            
            tissue_mesh_spl = o3d.io.read_triangle_mesh(tissue_path_spl)
            cells_mesh_spl = o3d.io.read_triangle_mesh(cells_path_spl)
            nuclei_mesh_spl = o3d.io.read_triangle_mesh(nuclei_path_spl)

            self.tissue_meshes_myo.append(tissue_mesh_myo)
            self.cells_meshes_myo.append(cells_mesh_myo)
            self.nuclei_meshes_myo.append(nuclei_mesh_myo)
            
            self.tissue_meshes_spl.append(tissue_mesh_spl)
            self.cells_meshes_spl.append(cells_mesh_spl)
            self.nuclei_meshes_spl.append(nuclei_mesh_spl)
            
            land_path = v.data_path + f'Landmarks/2019{specimen}_key_points.json'
            with open(land_path, 'r') as f:
                key_points = json.load(f)
                self.landmarks.append({
                    'myo_myo': {
                        name: key_points[name] 
                        for name in v.myo_myo_landmark_names 
                        if name in key_points
                    },
                    'spl_spl': {
                        name: key_points[name] 
                        for name in v.spl_spl_landmark_names 
                        if name in key_points
                    },
                    'myo_spl': {
                        name: key_points[name] 
                        for name in v.myo_spl_landmark_names 
                        if name in key_points
                    },
                })
            
            assert len(self.tissue_meshes_myo) == len(self.cells_meshes_myo) == len(self.nuclei_meshes_myo), f'Meshes not loaded correctly, tissue: {len(self.tissue_meshes_myo)}, cells: {len(self.cells_meshes_myo)}, nuclei: {len(self.nuclei_meshes_myo)}'
        
    def normalize_mesh_scale(self, mesh_idx, target_scale=1.0, tissue='myocardium'):
        if tissue == 'myocardium':
            tissue_mesh = self.tissue_meshes_myo[mesh_idx]
            cells_mesh = self.cells_meshes_myo[mesh_idx]
            nuclei_mesh = self.nuclei_meshes_myo[mesh_idx]
        else:
            tissue_mesh = self.tissue_meshes_spl[mesh_idx]
            cells_mesh = self.cells_meshes_spl[mesh_idx]
            nuclei_mesh = self.nuclei_meshes_spl[mesh_idx]
        
        bbox = tissue_mesh.get_axis_aligned_bounding_box()
        max_extent = np.max(bbox.get_extent())
        scale_factor = target_scale / max_extent
        
        center = tissue_mesh.get_center()
        tissue_mesh.scale(scale_factor, center=center)
        cells_mesh.scale(scale_factor, center=center)
        nuclei_mesh.scale(scale_factor, center=center)
    
    def center_mesh_at_origin(self, mesh_idx, tissue='myocardium'):
        if tissue == 'myocardium':
            tissue_mesh = self.tissue_meshes_myo[mesh_idx]
            cells_mesh = self.cells_meshes_myo[mesh_idx]
            nuclei_mesh = self.nuclei_meshes_myo[mesh_idx]
        else:
            tissue_mesh = self.tissue_meshes_spl[mesh_idx]
            cells_mesh = self.cells_meshes_spl[mesh_idx]
            nuclei_mesh = self.nuclei_meshes_spl[mesh_idx]
        
        centroid = tissue_mesh.get_center()
        
        tissue_mesh.translate(-centroid)
        cells_mesh.translate(-centroid)
        nuclei_mesh.translate(-centroid)
        

In [3]:
def show(mr):        
    for mesh in mr.tissue_meshes_myo + mr.tissue_meshes_spl:
        mesh.paint_uniform_color([1.0, 0.0, 0.0])  # Red color for tissues
    
    for mesh in mr.cells_meshes_myo + mr.cells_meshes_spl:
        mesh.paint_uniform_color([0.0, 1.0, 0.0])  # Green color for cells
    
    for mesh in mr.nuclei_meshes_myo + mr.nuclei_meshes_spl:
        mesh.paint_uniform_color([0.0, 0.0, 1.0])  # Blue color for nuclei
        
    all_meshes = mr.tissue_meshes_myo + mr.cells_meshes_myo + mr.nuclei_meshes_myo + mr.tissue_meshes_spl + mr.cells_meshes_spl + mr.nuclei_meshes_spl

    # Compute normals for shading
    for mesh in all_meshes:
        mesh.compute_vertex_normals()
        
    o3d.visualization.draw_geometries(all_meshes)

In [4]:
mr = MeshRegistration(group='Gr4')

In [22]:
idx = 0

t_m = mr.tissue_meshes_myo[idx] 
t_m.paint_uniform_color([1.0, 0.0, 0.0])
t_m.compute_vertex_normals()

t_s = mr.tissue_meshes_spl[idx]
t_s.paint_uniform_color([0.0, 1.0, 0.0])
t_s.compute_vertex_normals()

l_m = mr.landmarks[idx]['myo_myo'] 
l_s = mr.landmarks[idx]['spl_spl']

# create empty mesh
l = o3d.geometry.TriangleMesh()

for name, point in list(l_m.items()) + list(l_s.items()):
    p = o3d.geometry.TriangleMesh.create_sphere(radius=7)
    p.paint_uniform_color([0.0, 0.0, 1.0])
    p.translate(point)
    l += p
    
o3d.visualization.draw_geometries([t_m, t_s, l])

In [368]:
gr = 'Gr4'
specimens = v.specimens[gr]

spec_idx = 0
s = specimens[spec_idx]

myo_path = v.data_path + f'{gr}/3DShape/Tissue/myocardium/2019{s}Shape.ply'
spl_path = v.data_path + f'{gr}/3DShape/Tissue/splanchnic/2019{s}Shape.ply'

myo_cells_path = v.data_path + f'{gr}/3DShape/Membrane/myocardium/2019{s}_myocardium.ply'
spl_cells_path = v.data_path + f'{gr}/3DShape/Membrane/splanchnic/2019{s}_splanchnic.ply'

myo_mesh = o3d.io.read_triangle_mesh(myo_path)
spl_mesh = o3d.io.read_triangle_mesh(spl_path)

myo_cells_mesh = o3d.io.read_triangle_mesh(myo_cells_path)
spl_cells_mesh = o3d.io.read_triangle_mesh(spl_cells_path)

myo_mesh.paint_uniform_color([1.0, 0.0, 0.0])
spl_mesh.paint_uniform_color([0.0, 1.0, 0.0])

myo_cells_mesh.paint_uniform_color([1.0, 0.0, 0.0])
spl_cells_mesh.paint_uniform_color([0.0, 1.0, 0.0])

myo_mesh.compute_vertex_normals()
spl_mesh.compute_vertex_normals()

myo_cells_mesh.compute_vertex_normals()
spl_cells_mesh.compute_vertex_normals()

land_path = v.data_path + f'Landmarks/2019{s}_key_points.json'
with open(land_path, 'r') as f:
    key_points = json.load(f)
    myo_landmarks = {
        name: key_points[name] 
        for name in v.myo_myo_landmark_names 
        if name in key_points
    }
    spl_landmarks = {
        name: key_points[name] 
        for name in v.spl_spl_landmark_names 
        if name in key_points
    }
    common_landmarks = {
        name: key_points[name]
        for name in v.myo_spl_landmark_names
        if name in key_points
    }
    
l_m = o3d.geometry.TriangleMesh()

for name, point in myo_landmarks.items():
    p = o3d.geometry.TriangleMesh.create_sphere(radius=7)
    p.paint_uniform_color([1.0, 0.0, 1.0])
    p.translate(point)
    l_m += p
    
l_s = o3d.geometry.TriangleMesh()

for name, point in spl_landmarks.items():
    p = o3d.geometry.TriangleMesh.create_sphere(radius=7)
    p.paint_uniform_color([1.0, 1.0, 0.0])
    p.translate(point)
    l_s += p
    
o3d.visualization.draw_geometries([
    myo_mesh, spl_mesh, 
    # myo_cells_mesh, spl_cells_mesh,
    l_m, l_s
])

print(gr, s)

Gr4 0521_E1


In [363]:
import math

center = myo_mesh.get_center()

mirror = np.array(
    [[1, 0, 0, 0],
     [0, -1, 0, 0], # 60 
     [0, 0, 1, 0],
     [0, 0, 0, 1]]
)

rotation_angle = math.radians(20)  # Example: 15 degrees

# Create the rotation matrix for rotating around the Z-axis
# mirror = np.array([
#     [math.cos(rotation_angle), -math.sin(rotation_angle), 0, 0],
#     [math.sin(rotation_angle),  math.cos(rotation_angle), 0, 0],
#     [0, 0, 1, 0],
#     [0, 0, 0, 1]
# ])

# # Create the rotation matrix for rotating around the Y-axis
# mirror = np.array([
#     [math.cos(rotation_angle), 0, math.sin(rotation_angle), 0],
#     [0, 1, 0, 0],
#     [-math.sin(rotation_angle), 0, math.cos(rotation_angle), 0],
#     [0, 0, 0, 1]
# ])

# Create the rotation matrix for rotating around the X-axis
# mirror = np.array([
#     [1, 0, 0, 0],
#     [0, math.cos(rotation_angle), -math.sin(rotation_angle), 0],
#     [0, math.sin(rotation_angle), math.cos(rotation_angle), 0],
#     [0, 0, 0, 1]
# ])

trans2origin = np.eye(4)
trans2origin[:3, 3] = -center

back2center = np.eye(4)
back2center[:3, 3] = center

trans = back2center @ mirror @ trans2origin

myo_mesh.transform(trans)
spl_mesh.transform(trans)

o3d.visualization.draw_geometries([
    myo_mesh, spl_mesh, 
    # myo_cells_mesh, spl_cells_mesh,
    l_m, l_s
])

In [364]:
o3d.visualization.draw_geometries([
    myo_mesh, spl_mesh, 
    myo_cells_mesh, spl_cells_mesh,
    l_m, l_s
])

In [72]:
# center = myo_cells_mesh.get_center()
# 
# mirror = np.array(
#     [[1, 0, 0, 10],
#      [0, 1, 0, 0], # 60 
#      [0, 0, 1, 0],
#      [0, 0, 0, 1]]
# )
# 
# trans2origin = np.eye(4)
# trans2origin[:3, 3] = -center
# 
# back2center = np.eye(4)
# back2center[:3, 3] = center
# 
# trans = back2center @ mirror @ trans2origin
# 
# myo_cells_mesh.transform(trans)
# spl_cells_mesh.transform(trans)
# 
# o3d.visualization.draw_geometries([
#     myo_mesh, spl_mesh, 
#     myo_cells_mesh, spl_cells_mesh,
#     l_m, l_s
# ])

In [365]:
o3d.io.write_triangle_mesh(myo_path, myo_mesh)
o3d.io.write_triangle_mesh(spl_path, spl_mesh)

True

In [74]:
# o3d.io.write_triangle_mesh(myo_cells_path, myo_cells_mesh)
# o3d.io.write_triangle_mesh(spl_cells_path, spl_cells_mesh)

True

In [22]:
for i in range(len(mr.specimens)):
    for tissue in ['myocardium', 'splanchnic']:
        mr.normalize_mesh_scale(i, tissue=tissue)
        mr.center_mesh_at_origin(i, tissue=tissue)
    

In [24]:
show(mr)